# Forms and Admin Interface

## Learning Objectives

By the end of this notebook, you will be able to:

1. Create and configure Django forms
2. Understand form fields and validation
3. Use ModelForms to create forms from models
4. Handle form submissions in views
5. Implement CSRF protection
6. Set up and configure the Django admin
7. Register models and customize the admin interface

---

## 1. Introduction to Django Forms

Django forms handle:
- **Rendering HTML** form elements
- **Validating** submitted data
- **Converting** data to Python types
- **Displaying** error messages

Forms provide a secure, reusable way to handle user input.

## 2. Creating Basic Forms

Forms are defined in a `forms.py` file:

```python
# catalog/forms.py
from django import forms

class ContactForm(forms.Form):
    """Contact form for customer inquiries."""
    name = forms.CharField(
        max_length=100,
        widget=forms.TextInput(attrs={
            'class': 'form-control',
            'placeholder': 'Your name'
        })
    )
    email = forms.EmailField(
        widget=forms.EmailInput(attrs={
            'class': 'form-control',
            'placeholder': 'your@email.com'
        })
    )
    subject = forms.CharField(max_length=200)
    message = forms.CharField(
        widget=forms.Textarea(attrs={
            'class': 'form-control',
            'rows': 5
        })
    )
    
    # Optional field
    phone = forms.CharField(max_length=20, required=False)
```

## 3. Form Fields

Django provides many built-in field types:

In [ ]:
# Common form field types (for reference)

form_fields = {
    'Text Fields': {
        'CharField': 'Single line text (requires max_length)',
        'TextField': 'Multi-line text (use Textarea widget)',
        'EmailField': 'Email with validation',
        'URLField': 'URL with validation',
        'SlugField': 'Slug (letters, numbers, hyphens)',
        'RegexField': 'Text matching a regex pattern',
    },
    'Numeric Fields': {
        'IntegerField': 'Integer value',
        'FloatField': 'Floating point number',
        'DecimalField': 'Fixed precision decimal',
    },
    'Choice Fields': {
        'ChoiceField': 'Select dropdown (single choice)',
        'MultipleChoiceField': 'Select multiple options',
        'TypedChoiceField': 'Choice with type coercion',
    },
    'Boolean Fields': {
        'BooleanField': 'Checkbox (required=True means must be checked)',
        'NullBooleanField': 'Yes/No/Unknown dropdown',
    },
    'Date/Time Fields': {
        'DateField': 'Date input',
        'TimeField': 'Time input',
        'DateTimeField': 'Date and time',
        'DurationField': 'Time duration',
    },
    'File Fields': {
        'FileField': 'File upload',
        'ImageField': 'Image upload (validates image)',
    },
}

for category, fields in form_fields.items():
    print(f"\n{category}:")
    print("-" * 40)
    for name, desc in fields.items():
        print(f"  {name:25} - {desc}")

### Field Options

```python
from django import forms

class ExampleForm(forms.Form):
    # Required field (default)
    name = forms.CharField(max_length=100)
    
    # Optional field
    nickname = forms.CharField(max_length=50, required=False)
    
    # With default value
    country = forms.CharField(max_length=50, initial='USA')
    
    # With help text
    email = forms.EmailField(
        help_text='We will never share your email.'
    )
    
    # With custom label
    dob = forms.DateField(label='Date of Birth')
    
    # With custom error messages
    age = forms.IntegerField(
        min_value=18,
        max_value=120,
        error_messages={
            'min_value': 'You must be at least 18 years old.',
            'max_value': 'Please enter a valid age.',
        }
    )
    
    # Choice field
    PRIORITY_CHOICES = [
        ('', 'Select priority'),  # Empty default
        ('low', 'Low'),
        ('medium', 'Medium'),
        ('high', 'High'),
    ]
    priority = forms.ChoiceField(choices=PRIORITY_CHOICES)
    
    # Boolean (checkbox)
    agree_terms = forms.BooleanField(
        label='I agree to the terms and conditions',
        error_messages={
            'required': 'You must agree to the terms to continue.'
        }
    )
```

## 4. Widgets

Widgets control how form fields are rendered as HTML:

```python
from django import forms

class StyledForm(forms.Form):
    # Text input with attributes
    username = forms.CharField(
        widget=forms.TextInput(attrs={
            'class': 'form-control',
            'placeholder': 'Enter username',
            'autofocus': True,
        })
    )
    
    # Password input
    password = forms.CharField(
        widget=forms.PasswordInput(attrs={'class': 'form-control'})
    )
    
    # Textarea
    bio = forms.CharField(
        widget=forms.Textarea(attrs={
            'class': 'form-control',
            'rows': 4,
            'cols': 50,
        })
    )
    
    # Select dropdown
    category = forms.ChoiceField(
        choices=[('fiction', 'Fiction'), ('non-fiction', 'Non-Fiction')],
        widget=forms.Select(attrs={'class': 'form-select'})
    )
    
    # Radio buttons
    gender = forms.ChoiceField(
        choices=[('m', 'Male'), ('f', 'Female'), ('o', 'Other')],
        widget=forms.RadioSelect
    )
    
    # Multiple checkboxes
    interests = forms.MultipleChoiceField(
        choices=[
            ('tech', 'Technology'),
            ('sports', 'Sports'),
            ('music', 'Music'),
        ],
        widget=forms.CheckboxSelectMultiple
    )
    
    # Hidden input
    referrer = forms.CharField(
        widget=forms.HiddenInput,
        required=False
    )
    
    # Date picker (HTML5)
    birth_date = forms.DateField(
        widget=forms.DateInput(attrs={
            'type': 'date',
            'class': 'form-control'
        })
    )
    
    # Number input
    quantity = forms.IntegerField(
        widget=forms.NumberInput(attrs={
            'class': 'form-control',
            'min': 1,
            'max': 100,
        })
    )
```

## 5. Form Validation

### Built-in Validation

Fields have automatic validation:

```python
class RegistrationForm(forms.Form):
    username = forms.CharField(
        min_length=3,
        max_length=20,
    )
    email = forms.EmailField()  # Validates email format
    age = forms.IntegerField(
        min_value=13,
        max_value=120,
    )
    website = forms.URLField(required=False)  # Validates URL format
```

### Custom Field Validation

```python
from django import forms
from django.core.exceptions import ValidationError

class RegistrationForm(forms.Form):
    username = forms.CharField(max_length=20)
    email = forms.EmailField()
    password = forms.CharField(widget=forms.PasswordInput)
    confirm_password = forms.CharField(widget=forms.PasswordInput)
    
    def clean_username(self):
        """Validate individual field (clean_<fieldname>)."""
        username = self.cleaned_data['username']
        
        # Check for reserved words
        reserved = ['admin', 'root', 'system']
        if username.lower() in reserved:
            raise ValidationError('This username is reserved.')
        
        # Check if username exists (would need to import User model)
        # if User.objects.filter(username=username).exists():
        #     raise ValidationError('Username already taken.')
        
        return username
    
    def clean_email(self):
        """Validate email domain."""
        email = self.cleaned_data['email']
        
        blocked_domains = ['tempmail.com', 'throwaway.com']
        domain = email.split('@')[1]
        if domain in blocked_domains:
            raise ValidationError('Please use a permanent email address.')
        
        return email
    
    def clean(self):
        """Validate multiple fields together."""
        cleaned_data = super().clean()
        password = cleaned_data.get('password')
        confirm_password = cleaned_data.get('confirm_password')
        
        if password and confirm_password:
            if password != confirm_password:
                raise ValidationError('Passwords do not match.')
            
            if len(password) < 8:
                self.add_error('password', 'Password must be at least 8 characters.')
        
        return cleaned_data
```

## 6. ModelForms

ModelForms automatically create forms from models:

```python
# catalog/forms.py
from django import forms
from .models import Book, Review

class BookForm(forms.ModelForm):
    """Form for creating/editing books."""
    
    class Meta:
        model = Book
        fields = ['title', 'author', 'isbn', 'price', 'published_date', 'description']
        # Or use: fields = '__all__' for all fields
        # Or use: exclude = ['created_at'] to exclude specific fields
        
        widgets = {
            'title': forms.TextInput(attrs={'class': 'form-control'}),
            'description': forms.Textarea(attrs={'class': 'form-control', 'rows': 4}),
            'published_date': forms.DateInput(attrs={'type': 'date', 'class': 'form-control'}),
            'price': forms.NumberInput(attrs={'class': 'form-control', 'step': '0.01'}),
        }
        
        labels = {
            'isbn': 'ISBN',
            'published_date': 'Publication Date',
        }
        
        help_texts = {
            'isbn': 'Enter the 13-digit ISBN.',
        }
        
        error_messages = {
            'title': {
                'max_length': 'Title is too long.',
                'required': 'Please enter a title.',
            },
        }
    
    def clean_isbn(self):
        """Validate ISBN format."""
        isbn = self.cleaned_data['isbn']
        # Remove hyphens
        isbn = isbn.replace('-', '')
        if len(isbn) != 13:
            raise forms.ValidationError('ISBN must be 13 digits.')
        if not isbn.isdigit():
            raise forms.ValidationError('ISBN must contain only digits.')
        return isbn


class ReviewForm(forms.ModelForm):
    """Form for submitting book reviews."""
    
    class Meta:
        model = Review
        fields = ['rating', 'comment']
        widgets = {
            'rating': forms.RadioSelect(choices=[
                (1, '1 star'),
                (2, '2 stars'),
                (3, '3 stars'),
                (4, '4 stars'),
                (5, '5 stars'),
            ]),
            'comment': forms.Textarea(attrs={'rows': 3}),
        }
```

## 7. Handling Forms in Views

### Function-Based View

```python
# catalog/views.py
from django.shortcuts import render, redirect, get_object_or_404
from django.contrib import messages
from .forms import BookForm, ContactForm
from .models import Book

def create_book(request):
    """Create a new book."""
    if request.method == 'POST':
        form = BookForm(request.POST)
        if form.is_valid():
            book = form.save()
            messages.success(request, f'Book "{book.title}" created successfully!')
            return redirect('catalog:book_detail', book_id=book.id)
    else:
        form = BookForm()
    
    return render(request, 'catalog/book_form.html', {'form': form})


def edit_book(request, book_id):
    """Edit an existing book."""
    book = get_object_or_404(Book, pk=book_id)
    
    if request.method == 'POST':
        form = BookForm(request.POST, instance=book)
        if form.is_valid():
            form.save()
            messages.success(request, 'Book updated successfully!')
            return redirect('catalog:book_detail', book_id=book.id)
    else:
        form = BookForm(instance=book)
    
    return render(request, 'catalog/book_form.html', {
        'form': form,
        'book': book,
        'editing': True,
    })


def contact(request):
    """Handle contact form (non-ModelForm)."""
    if request.method == 'POST':
        form = ContactForm(request.POST)
        if form.is_valid():
            # Access cleaned data
            name = form.cleaned_data['name']
            email = form.cleaned_data['email']
            message = form.cleaned_data['message']
            
            # Process the form (e.g., send email)
            # send_mail(...)
            
            messages.success(request, 'Thank you for your message!')
            return redirect('catalog:contact_success')
    else:
        form = ContactForm()
    
    return render(request, 'catalog/contact.html', {'form': form})
```

### Class-Based View with Forms

```python
from django.views.generic import CreateView, UpdateView
from django.urls import reverse_lazy
from django.contrib.messages.views import SuccessMessageMixin
from .models import Book
from .forms import BookForm

class BookCreateView(SuccessMessageMixin, CreateView):
    """Create a new book using CBV."""
    model = Book
    form_class = BookForm
    template_name = 'catalog/book_form.html'
    success_url = reverse_lazy('catalog:book_list')
    success_message = 'Book "%(title)s" created successfully!'
    
    def form_valid(self, form):
        """Called when form is valid."""
        form.instance.created_by = self.request.user
        return super().form_valid(form)


class BookUpdateView(SuccessMessageMixin, UpdateView):
    """Edit an existing book using CBV."""
    model = Book
    form_class = BookForm
    template_name = 'catalog/book_form.html'
    pk_url_kwarg = 'book_id'
    success_message = 'Book updated successfully!'
    
    def get_success_url(self):
        return reverse_lazy('catalog:book_detail', kwargs={'book_id': self.object.pk})
```

## 8. Rendering Forms in Templates

```html
<!-- catalog/templates/catalog/book_form.html -->
{% extends 'base.html' %}

{% block content %}
<h1>{% if editing %}Edit{% else %}Add{% endif %} Book</h1>

<form method="post" novalidate>
    {% csrf_token %}
    
    {# Option 1: Render entire form #}
    {{ form.as_p }}
    
    {# Other options: form.as_table, form.as_ul #}
    
    <button type="submit">Save</button>
</form>
{% endblock %}
```

### Manual Field Rendering

```html
{% extends 'base.html' %}

{% block content %}
<h1>{% if editing %}Edit{% else %}Add{% endif %} Book</h1>

<form method="post" novalidate>
    {% csrf_token %}
    
    {# Display non-field errors #}
    {% if form.non_field_errors %}
        <div class="alert alert-danger">
            {{ form.non_field_errors }}
        </div>
    {% endif %}
    
    {# Render each field manually #}
    <div class="form-group {% if form.title.errors %}has-error{% endif %}">
        <label for="{{ form.title.id_for_label }}">{{ form.title.label }}</label>
        {{ form.title }}
        {% if form.title.help_text %}
            <small class="help-text">{{ form.title.help_text }}</small>
        {% endif %}
        {% for error in form.title.errors %}
            <span class="error">{{ error }}</span>
        {% endfor %}
    </div>
    
    <div class="form-group">
        <label for="{{ form.author.id_for_label }}">{{ form.author.label }}</label>
        {{ form.author }}
        {% for error in form.author.errors %}
            <span class="error">{{ error }}</span>
        {% endfor %}
    </div>
    
    {# Or loop through all fields #}
    {% for field in form %}
        <div class="form-group {% if field.errors %}has-error{% endif %}">
            <label for="{{ field.id_for_label }}">{{ field.label }}</label>
            {{ field }}
            {% if field.help_text %}
                <small>{{ field.help_text }}</small>
            {% endif %}
            {% for error in field.errors %}
                <span class="error">{{ error }}</span>
            {% endfor %}
        </div>
    {% endfor %}
    
    <button type="submit" class="btn btn-primary">Save</button>
    <a href="{% url 'catalog:book_list' %}" class="btn btn-secondary">Cancel</a>
</form>
{% endblock %}
```

## 9. CSRF Protection

Django protects against Cross-Site Request Forgery (CSRF) attacks.

### Always Include CSRF Token

```html
<form method="post">
    {% csrf_token %}  <!-- Required for POST forms -->
    {{ form.as_p }}
    <button type="submit">Submit</button>
</form>
```

### For AJAX Requests

```javascript
// Get CSRF token from cookie
function getCookie(name) {
    let cookieValue = null;
    if (document.cookie && document.cookie !== '') {
        const cookies = document.cookie.split(';');
        for (let i = 0; i < cookies.length; i++) {
            const cookie = cookies[i].trim();
            if (cookie.substring(0, name.length + 1) === (name + '=')) {
                cookieValue = decodeURIComponent(cookie.substring(name.length + 1));
                break;
            }
        }
    }
    return cookieValue;
}

const csrftoken = getCookie('csrftoken');

// Include in fetch requests
fetch('/api/books/', {
    method: 'POST',
    headers: {
        'Content-Type': 'application/json',
        'X-CSRFToken': csrftoken,
    },
    body: JSON.stringify(data),
});
```

### Exempt a View (Use Carefully!)

```python
from django.views.decorators.csrf import csrf_exempt

@csrf_exempt
def webhook_receiver(request):
    """Receive webhooks from external services."""
    # Only exempt when absolutely necessary (e.g., webhooks)
    pass
```

## 10. File Uploads

```python
# catalog/forms.py
from django import forms
from .models import Book

class BookWithCoverForm(forms.ModelForm):
    class Meta:
        model = Book
        fields = ['title', 'author', 'cover_image']
    
    def clean_cover_image(self):
        image = self.cleaned_data.get('cover_image')
        if image:
            # Check file size (e.g., max 5MB)
            if image.size > 5 * 1024 * 1024:
                raise forms.ValidationError('Image must be less than 5MB.')
            
            # Check file type
            valid_types = ['image/jpeg', 'image/png', 'image/gif']
            if image.content_type not in valid_types:
                raise forms.ValidationError('Only JPEG, PNG, and GIF are allowed.')
        return image
```

```python
# catalog/views.py
def upload_cover(request, book_id):
    book = get_object_or_404(Book, pk=book_id)
    
    if request.method == 'POST':
        # Note: request.FILES for file uploads
        form = BookWithCoverForm(request.POST, request.FILES, instance=book)
        if form.is_valid():
            form.save()
            return redirect('catalog:book_detail', book_id=book.id)
    else:
        form = BookWithCoverForm(instance=book)
    
    return render(request, 'catalog/upload_cover.html', {'form': form})
```

```html
<!-- Note: enctype for file uploads -->
<form method="post" enctype="multipart/form-data">
    {% csrf_token %}
    {{ form.as_p }}
    <button type="submit">Upload</button>
</form>
```

## 11. Django Admin Setup

The Django admin is a powerful built-in interface for managing your data.

### Enable Admin

```python
# mysite/settings.py
INSTALLED_APPS = [
    'django.contrib.admin',  # Already included by default
    'django.contrib.auth',
    'django.contrib.contenttypes',
    'django.contrib.sessions',
    'django.contrib.messages',
    'django.contrib.staticfiles',
    'catalog',
]
```

```python
# mysite/urls.py
from django.contrib import admin
from django.urls import path, include

urlpatterns = [
    path('admin/', admin.site.urls),  # Admin URL
    path('catalog/', include('catalog.urls')),
]
```

### Create Superuser

```bash
python manage.py createsuperuser
# Enter username, email, and password
```

## 12. Registering Models with Admin

```python
# catalog/admin.py
from django.contrib import admin
from .models import Author, Book, Category, Review

# Simple registration
admin.site.register(Category)

# Registration with customization
@admin.register(Author)
class AuthorAdmin(admin.ModelAdmin):
    list_display = ['last_name', 'first_name', 'birth_date']
    list_filter = ['birth_date']
    search_fields = ['first_name', 'last_name']
    ordering = ['last_name', 'first_name']


@admin.register(Book)
class BookAdmin(admin.ModelAdmin):
    # Display in list view
    list_display = ['title', 'author', 'price', 'in_stock', 'published_date']
    
    # Filters in sidebar
    list_filter = ['in_stock', 'published_date', 'category']
    
    # Search functionality
    search_fields = ['title', 'isbn', 'author__last_name']
    
    # Default ordering
    ordering = ['-published_date']
    
    # Items per page
    list_per_page = 25
    
    # Editable in list view
    list_editable = ['price', 'in_stock']
    
    # Date hierarchy navigation
    date_hierarchy = 'published_date'
    
    # Prepopulate slug from title
    prepopulated_fields = {'slug': ('title',)}
```

## 13. Customizing Admin Interface

### Fieldsets (Organize Edit Form)

```python
@admin.register(Book)
class BookAdmin(admin.ModelAdmin):
    list_display = ['title', 'author', 'price', 'in_stock']
    
    fieldsets = [
        (None, {
            'fields': ['title', 'slug', 'author']
        }),
        ('Publication Info', {
            'fields': ['isbn', 'published_date', 'category']
        }),
        ('Pricing & Stock', {
            'fields': ['price', 'in_stock'],
            'classes': ['collapse'],  # Collapsible section
        }),
        ('Description', {
            'fields': ['description'],
            'classes': ['wide'],
        }),
    ]
    
    # Read-only fields
    readonly_fields = ['created_at', 'updated_at']
    
    # Autocomplete for ForeignKey (requires search_fields on related model)
    autocomplete_fields = ['author']
    
    # Raw ID widget (for large datasets)
    raw_id_fields = ['author']
```

### Inline Related Models

```python
class ReviewInline(admin.TabularInline):
    """Display reviews inline on book page."""
    model = Review
    extra = 1  # Number of empty forms to display
    readonly_fields = ['created_at']
    
    # Or use StackedInline for vertical layout
    # class ReviewInline(admin.StackedInline):
    #     model = Review


@admin.register(Book)
class BookAdmin(admin.ModelAdmin):
    list_display = ['title', 'author', 'price']
    inlines = [ReviewInline]
```

### Custom Admin Actions

```python
@admin.register(Book)
class BookAdmin(admin.ModelAdmin):
    list_display = ['title', 'author', 'price', 'in_stock']
    actions = ['mark_in_stock', 'mark_out_of_stock', 'apply_discount']
    
    @admin.action(description='Mark selected books as in stock')
    def mark_in_stock(self, request, queryset):
        count = queryset.update(in_stock=True)
        self.message_user(request, f'{count} books marked as in stock.')
    
    @admin.action(description='Mark selected books as out of stock')
    def mark_out_of_stock(self, request, queryset):
        count = queryset.update(in_stock=False)
        self.message_user(request, f'{count} books marked as out of stock.')
    
    @admin.action(description='Apply 10% discount')
    def apply_discount(self, request, queryset):
        from decimal import Decimal
        for book in queryset:
            book.price = book.price * Decimal('0.90')
            book.save()
        self.message_user(request, f'10% discount applied to {queryset.count()} books.')
```

### Custom Admin Display

```python
from django.utils.html import format_html

@admin.register(Book)
class BookAdmin(admin.ModelAdmin):
    list_display = ['title', 'author', 'colored_price', 'stock_status', 'cover_thumbnail']
    
    @admin.display(description='Price', ordering='price')
    def colored_price(self, obj):
        """Display price with color based on value."""
        if obj.price > 50:
            color = 'red'
        elif obj.price > 20:
            color = 'orange'
        else:
            color = 'green'
        return format_html(
            '<span style="color: {};">${}</span>',
            color,
            obj.price
        )
    
    @admin.display(description='Status', boolean=True)
    def stock_status(self, obj):
        """Display boolean as icon."""
        return obj.in_stock
    
    @admin.display(description='Cover')
    def cover_thumbnail(self, obj):
        """Display cover image thumbnail."""
        if obj.cover_image:
            return format_html(
                '<img src="{}" width="50" height="70" />',
                obj.cover_image.url
            )
        return 'No image'
```

### Customize Admin Site

```python
# catalog/admin.py
from django.contrib import admin

# Customize admin site
admin.site.site_header = 'Bookstore Administration'
admin.site.site_title = 'Bookstore Admin'
admin.site.index_title = 'Welcome to the Bookstore Admin'
```

---

## Exercises

### Exercise 1: Create a Contact Form

Create a ContactForm with fields for name, email, subject, and message. Add validation to ensure the message is at least 20 characters long.

<details>
<summary>Click to see solution</summary>

```python
# catalog/forms.py
from django import forms
from django.core.exceptions import ValidationError

class ContactForm(forms.Form):
    """Contact form with validation."""
    name = forms.CharField(
        max_length=100,
        widget=forms.TextInput(attrs={
            'class': 'form-control',
            'placeholder': 'Your name'
        })
    )
    email = forms.EmailField(
        widget=forms.EmailInput(attrs={
            'class': 'form-control',
            'placeholder': 'your@email.com'
        })
    )
    subject = forms.CharField(
        max_length=200,
        widget=forms.TextInput(attrs={
            'class': 'form-control',
            'placeholder': 'Subject'
        })
    )
    message = forms.CharField(
        widget=forms.Textarea(attrs={
            'class': 'form-control',
            'rows': 5,
            'placeholder': 'Your message (minimum 20 characters)'
        })
    )
    
    def clean_message(self):
        """Validate message length."""
        message = self.cleaned_data['message']
        if len(message) < 20:
            raise ValidationError(
                'Message must be at least 20 characters long. '
                f'You entered {len(message)} characters.'
            )
        return message
```

</details>

### Exercise 2: Create a ModelForm for Review

Create a ReviewForm ModelForm with fields for rating (1-5) and comment. Use RadioSelect for the rating.

<details>
<summary>Click to see solution</summary>

```python
# catalog/forms.py
from django import forms
from .models import Review

class ReviewForm(forms.ModelForm):
    """Form for submitting book reviews."""
    
    RATING_CHOICES = [
        (1, '1 - Poor'),
        (2, '2 - Fair'),
        (3, '3 - Good'),
        (4, '4 - Very Good'),
        (5, '5 - Excellent'),
    ]
    
    class Meta:
        model = Review
        fields = ['rating', 'reviewer_name', 'comment']
        widgets = {
            'rating': forms.RadioSelect(choices=RATING_CHOICES),
            'reviewer_name': forms.TextInput(attrs={
                'class': 'form-control',
                'placeholder': 'Your name'
            }),
            'comment': forms.Textarea(attrs={
                'class': 'form-control',
                'rows': 4,
                'placeholder': 'Write your review here...'
            }),
        }
        labels = {
            'reviewer_name': 'Your Name',
        }
```

</details>

### Exercise 3: Handle Form in View

Create a view that handles the ReviewForm for adding a review to a book.

<details>
<summary>Click to see solution</summary>

```python
# catalog/views.py
from django.shortcuts import render, redirect, get_object_or_404
from django.contrib import messages
from .models import Book
from .forms import ReviewForm

def add_review(request, book_id):
    """Add a review to a book."""
    book = get_object_or_404(Book, pk=book_id)
    
    if request.method == 'POST':
        form = ReviewForm(request.POST)
        if form.is_valid():
            review = form.save(commit=False)
            review.book = book
            review.save()
            messages.success(request, 'Thank you for your review!')
            return redirect('catalog:book_detail', book_id=book.id)
    else:
        form = ReviewForm()
    
    return render(request, 'catalog/add_review.html', {
        'form': form,
        'book': book,
    })


# catalog/urls.py
# path('books/<int:book_id>/review/', views.add_review, name='add_review'),
```

```html
<!-- catalog/templates/catalog/add_review.html -->
{% extends 'base.html' %}

{% block title %}Review {{ book.title }}{% endblock %}

{% block content %}
<h1>Review: {{ book.title }}</h1>
<p>By {{ book.author }}</p>

<form method="post" novalidate>
    {% csrf_token %}
    
    {% for field in form %}
        <div class="form-group">
            <label>{{ field.label }}</label>
            {{ field }}
            {% for error in field.errors %}
                <span class="error">{{ error }}</span>
            {% endfor %}
        </div>
    {% endfor %}
    
    <button type="submit" class="btn btn-primary">Submit Review</button>
    <a href="{% url 'catalog:book_detail' book.id %}" class="btn btn-secondary">Cancel</a>
</form>
{% endblock %}
```

</details>

### Exercise 4: Customize Admin for Book Model

Create an admin configuration for the Book model with:
- List display: title, author, price, in_stock, published_date
- Filters: in_stock, category, published_date
- Search: title, ISBN, author name
- Inline reviews

<details>
<summary>Click to see solution</summary>

```python
# catalog/admin.py
from django.contrib import admin
from .models import Book, Author, Category, Review

class ReviewInline(admin.TabularInline):
    model = Review
    extra = 0
    readonly_fields = ['created_at']
    fields = ['reviewer_name', 'rating', 'comment', 'created_at']


@admin.register(Author)
class AuthorAdmin(admin.ModelAdmin):
    list_display = ['last_name', 'first_name', 'birth_date']
    search_fields = ['first_name', 'last_name']
    ordering = ['last_name']


@admin.register(Category)
class CategoryAdmin(admin.ModelAdmin):
    list_display = ['name']
    search_fields = ['name']


@admin.register(Book)
class BookAdmin(admin.ModelAdmin):
    list_display = ['title', 'author', 'price', 'in_stock', 'published_date', 'review_count']
    list_filter = ['in_stock', 'category', 'published_date']
    search_fields = ['title', 'isbn', 'author__first_name', 'author__last_name']
    ordering = ['-published_date']
    list_per_page = 20
    list_editable = ['in_stock', 'price']
    date_hierarchy = 'published_date'
    
    fieldsets = [
        ('Book Information', {
            'fields': ['title', 'author', 'isbn', 'category']
        }),
        ('Pricing & Availability', {
            'fields': ['price', 'in_stock']
        }),
        ('Details', {
            'fields': ['description', 'published_date'],
            'classes': ['collapse']
        }),
    ]
    
    inlines = [ReviewInline]
    autocomplete_fields = ['author']
    
    @admin.display(description='Reviews')
    def review_count(self, obj):
        return obj.reviews.count()
```

</details>

### Exercise 5: Admin Actions

Add custom admin actions to:
1. Mark selected books as "featured"
2. Export selected books to CSV

<details>
<summary>Click to see solution</summary>

```python
# catalog/admin.py
import csv
from django.http import HttpResponse
from django.contrib import admin
from .models import Book

@admin.register(Book)
class BookAdmin(admin.ModelAdmin):
    list_display = ['title', 'author', 'price', 'in_stock', 'is_featured']
    actions = ['mark_as_featured', 'unmark_as_featured', 'export_to_csv']
    
    @admin.action(description='Mark selected books as featured')
    def mark_as_featured(self, request, queryset):
        count = queryset.update(is_featured=True)
        self.message_user(
            request,
            f'{count} book(s) marked as featured.'
        )
    
    @admin.action(description='Remove featured status from selected books')
    def unmark_as_featured(self, request, queryset):
        count = queryset.update(is_featured=False)
        self.message_user(
            request,
            f'{count} book(s) removed from featured.'
        )
    
    @admin.action(description='Export selected books to CSV')
    def export_to_csv(self, request, queryset):
        response = HttpResponse(content_type='text/csv')
        response['Content-Disposition'] = 'attachment; filename="books.csv"'
        
        writer = csv.writer(response)
        writer.writerow(['Title', 'Author', 'ISBN', 'Price', 'In Stock', 'Published'])
        
        for book in queryset:
            writer.writerow([
                book.title,
                str(book.author),
                book.isbn,
                book.price,
                'Yes' if book.in_stock else 'No',
                book.published_date.strftime('%Y-%m-%d') if book.published_date else '',
            ])
        
        self.message_user(
            request,
            f'{queryset.count()} book(s) exported to CSV.'
        )
        return response
```

</details>

---

## Summary

In this notebook, you learned:

- **Django Forms** handle rendering, validation, and data cleaning
- **Form fields** provide various input types with built-in validation
- **Widgets** control how fields render as HTML
- **ModelForms** automatically create forms from models
- **Validation** can be customized with `clean_<fieldname>()` and `clean()` methods
- **CSRF protection** is required for all POST forms
- **Django Admin** provides a powerful interface for managing data
- **Admin customization** includes list display, filters, inline models, and actions

## Next Steps

You've completed the Django module! Here are suggestions for further learning:

1. **Authentication**: User login, logout, registration, and permissions
2. **REST APIs**: Build APIs with Django REST Framework
3. **Testing**: Write unit tests for models, views, and forms
4. **Deployment**: Deploy Django apps with Gunicorn, Nginx, and PostgreSQL
5. **Caching**: Improve performance with Django's caching framework
6. **Signals**: React to events in your application
7. **Celery**: Handle asynchronous tasks and background jobs